In [5]:
lib = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/'
import sys
sys.path.append(lib)
from utilities.package_importing import *
import scipy.stats as stats

In [6]:
############################  Helper functions ############################
def col_fixed(df, wsheet, col_skip=0):
    for idx, col in enumerate(df):  # loop through all columns
        series = df[col]
        max_len = max((
            series.astype(str).map(len).max(),  # len of largest item
            len(str(series.name))  # len of column name/header
            )) + 1  # adding a little extra space
        wsheet.set_column(idx+col_skip, idx+col_skip, max_len)  # set column width

# Combine pvals
After all analyses have been done, for example you compare NR mucosal vs NR cutaneous; ranksum fraction clusters of immune/CD8/myeloids and ranksum fraction of GEPs, 
you can now create an Excel that combines all pvals sorted in one sheet and examine your result.
The file will be saved in the main folder of the anlyses.

#### Specify path of analysis and then run all other blocks:

In [32]:
# FOLDER_PATH = r'/storage/md_keren/shitay/outputs/response_analysis/subcohort_1.1.22/8.1.22/NR_mucosal_vs_R_cutaneous'
# FOLDER_PATH = r'/storage/md_keren/shitay/outputs/response_analysis/subcohort_1.1.22/8.1.22/NR_mucosal_vs_R_AND_NR_cutaneous'
FOLDER_PATH = r'/storage/md_keren/shitay/outputs/response_analysis/subcohort_1.1.22/8.1.22/NR_mucosal_vs_NR_cutaneous'

FILES_EXTENSION = 'csv' #

In [33]:
prefix_file_name = '_'.join(FOLDER_PATH.split('/')[-1].split('_')[:-1])
OUTPUT_FILE_NAME = fr'{prefix_file_name}_pvals_all_statistical_test_results.xlsx'

dfs paths (no need to change):

In [34]:
########################################### file paths ###########################################
GEP_path = f'statistical_tests/GEP_fraction_ranksum_test_pvals.{FILES_EXTENSION}'
immune_clustering_path = f'statistical_tests/cluster_fractions_ranksum_test_pvals.{FILES_EXTENSION}'
myeloid_clustering_path = f'myeloids/statistical_tests/cluster_fractions_ranksum_test_pvals.{FILES_EXTENSION}'
CD8_clustering_path = f'cytotoxic_t_cells/statistical_tests/cluster_fractions_ranksum_test_pvals.{FILES_EXTENSION}'

########################################### annotation paths ###########################################
immune_annotation_path = r'/storage/md_keren/shitay/Data/tables/clustering_annotations/for_pandas_df/Immune_clustering_26.6.21_annotations.xlsx'
myeloid_annotation_path = r'/storage/md_keren/shitay/Data/tables/clustering_annotations/for_pandas_df/Myeloid_clustering_K_11_11.8.21_annotations.xlsx'
CD8_annotation_path = r'/storage/md_keren/shitay/Data/tables/clustering_annotations/for_pandas_df/CD8_clustering_K_10_11.8.21_annotations.xlsx'
########################################### GEP pathways path ###########################################
GEP_pathways_path = r'/storage/md_keren/shitay/Data/tables/GEP/subcohort_1.1.22/k20_2000genes_200iter/GEPS_pathways.xlsx'

import xlsx files:

In [35]:
if FILES_EXTENSION=='csv':
    read_func = pd.read_csv
else:
    read_func = pd.read_excel
########################################### dfs ###########################################
GEP_df = read_func(join(FOLDER_PATH, GEP_path))
immune_clustering_df = read_func(join(FOLDER_PATH, immune_clustering_path))
myeloid_clustering_df = read_func(join(FOLDER_PATH, myeloid_clustering_path))
CD8_clustering_df = read_func(join(FOLDER_PATH, CD8_clustering_path))

########################################### annotation dfs ###########################################
immune_annotation_df = pd.read_excel(join(FOLDER_PATH, immune_annotation_path))
myeloid_annotation_df = pd.read_excel(join(FOLDER_PATH, myeloid_annotation_path))
CD8_annotation_df = pd.read_excel(join(FOLDER_PATH, CD8_annotation_path))
# GEP_pathways_df = pd.read_excel(GEP_pathways_path)

########################################### Add annotation to pval dfs ###########################################
immune_clustering_df['cluster annotation'] = immune_clustering_df['cluster'].apply(lambda x: immune_annotation_df.set_index('CLUSTER_IDX').loc[x]['annotations'])
myeloid_clustering_df['cluster annotation'] = myeloid_clustering_df['cluster'].apply(lambda x: myeloid_annotation_df.set_index('cluster').loc[x]['annotation'])
CD8_clustering_df['cluster annotation'] = CD8_clustering_df['cluster'].apply(lambda x: CD8_annotation_df.set_index('cluster').loc[x]['annotation'])

########################################### order pval dfs ###########################################
immune_clustering_df = immune_clustering_df[['cluster', 'cluster annotation', 'pval', 'corrected_pval', 'Median >']].sort_values('pval')
myeloid_clustering_df = myeloid_clustering_df[['cluster', 'cluster annotation', 'pval', 'corrected_pval', 'Median >']].sort_values('pval')
CD8_clustering_df = CD8_clustering_df[['cluster', 'cluster annotation', 'pval', 'corrected_pval', 'Median >']].sort_values('pval')
GEP_df = GEP_df.sort_values('pval')

########################################### round pval ###########################################
pval_round = 4
immune_clustering_df.pval = round(immune_clustering_df.pval, pval_round)
immune_clustering_df.corrected_pval = round(immune_clustering_df.corrected_pval, pval_round)

myeloid_clustering_df.pval = round(myeloid_clustering_df.pval, pval_round)
myeloid_clustering_df.corrected_pval = round(myeloid_clustering_df.corrected_pval, pval_round)

CD8_clustering_df.pval = round(CD8_clustering_df.pval, pval_round)
CD8_clustering_df.corrected_pval = round(CD8_clustering_df.corrected_pval, pval_round)

GEP_df.pval = round(GEP_df.pval, pval_round)
GEP_df.corrected_pval = round(GEP_df.corrected_pval, pval_round)

builds an Excel that will contain all statistical test results in one sheet:

In [36]:
GEP_df.name = "GEP fraction ranksum test (sorted by pval)"
immune_clustering_df.name = "Immune clustering fraction ranksum test (sorted by pval)"
myeloid_clustering_df.name = "myeloid clustering fraction ranksum test (sorted by pval)"
CD8_clustering_df.name = "CD8 clustering fraction ranksum test (sorted by pval)"

########################################### ExcelWriter ###########################################

writer = pd.ExcelWriter(join(FOLDER_PATH, OUTPUT_FILE_NAME),engine='xlsxwriter')
workbook=writer.book
worksheet=workbook.add_worksheet('Result')
writer.sheets['Result'] = worksheet

########################################### write header ###########################################

worksheet.write_string(0, 0, immune_clustering_df.name)
worksheet.write_string(immune_clustering_df.shape[0] + 4, 0, myeloid_clustering_df.name)

worksheet.write_string(0, immune_clustering_df.shape[1] + 1, CD8_clustering_df.name)
worksheet.write_string(immune_clustering_df.shape[0] + 4, immune_clustering_df.shape[1] + 1, GEP_df.name)

########################################### put dfs in xlsx ###########################################
immune_clustering_df.to_excel(writer,sheet_name='Result',startrow=1 , startcol=0, index=False)
col_fixed(immune_clustering_df, worksheet)

myeloid_clustering_df.to_excel(writer,sheet_name='Result',startrow=immune_clustering_df.shape[0] + 5, startcol=0, index=False)

CD8_clustering_df.to_excel(writer,sheet_name='Result',startrow=1, startcol=immune_clustering_df.shape[1] + 1, index=False)
col_fixed(CD8_clustering_df, worksheet, col_skip=immune_clustering_df.shape[1] + 1)

GEP_df.to_excel(writer,sheet_name='Result',startrow=immune_clustering_df.shape[0] + 5, startcol=immune_clustering_df.shape[1] + 1, index=False)

########################################### add GEP pathways ###########################################
pathways_col_skip = immune_clustering_df.shape[1]+ GEP_df.shape[1] + 3
n_p_in_row = 4
max_n_pathways = 8
for i in range(1, 21):
    if i==1:
        pathways = []
    else:
        row_pathways_df = pd.read_excel(GEP_pathways_path, f'GEP{i}')
        pathways = [v[9:].replace('_', ' ').lower() for v in row_pathways_df['Gene Set Name'].values.tolist()][:max_n_pathways]
    
    pr_pathways_df = pd.DataFrame(pathways,columns=[f'Pr{i}'])
    col_p = (i-1)%n_p_in_row
    row_p = int((i-1)/n_p_in_row)
    pr_pathways_df.to_excel(writer,sheet_name='Result',startrow=1 + row_p *  (max_n_pathways+2), startcol=pathways_col_skip + (col_p *2) + 1, index=False)
    
########################################### save ###########################################
writer.save()

# Debug

'NR_mucosal_vs_NR_cutaneous'